In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import plotly.express as px
from scipy.signal import argrelextrema

In [4]:
df = pd.read_csv("Stock.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.drop(columns=["Unnamed: 0"])
df.set_index("Date", inplace=True)

In [5]:
def weekly_price(x):
    weekly_price = df[x].resample("W").mean()
    local_max_index = argrelextrema(weekly_price.values, np.greater)[0]
    local_min_index = argrelextrema(weekly_price.values, np.less)[0]
    local_max_prices = weekly_price.iloc[local_max_index]
    local_min_prices = weekly_price.iloc[local_min_index]

    fig = px.line(weekly_price, title=f"Biểu Đồ Biến Động {x} Hàng Tuần", labels={'index': 'Ngày', 'value': f'Giá {x} (USD)'})
    fig.add_scatter(x=local_max_prices.index, y=local_max_prices, mode='markers', name='Điểm Cao', marker_symbol='triangle-up', marker_color='red')
    fig.add_scatter(x=local_min_prices.index, y=local_min_prices, mode='markers', name='Điểm Thấp', marker_symbol='triangle-down', marker_color='green')

    fig.show()

In [6]:
weekly_price('Nvidia_Price')

In [7]:
Nvidia=df[['Nvidia_Price','Nvidia_Vol.']]
Nvidia.columns = ['Price','Volume']
Nvidia['Trading_value']=Nvidia['Price']*Nvidia['Volume']
Nvidia['Daily_Return'] = Nvidia['Price'].pct_change()
Nvidia['30D MA'] = Nvidia['Price'].rolling(30).mean()
Nvidia['60D MA'] = Nvidia['Price'].rolling(60).mean()
Nvidia['Uper Band'] = Nvidia['Price'].rolling(30).mean() + 2*Nvidia['Price'].rolling(30).std()
Nvidia['Lower Band'] = Nvidia['Price'].rolling(30).mean() - 2*Nvidia['Price'].rolling(30).std()

C:\Users\Admin\AppData\Local\Temp\ipykernel_10724\2369707605.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_10724\2369707605.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_10724\2369707605.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [8]:
Nvidia.head()
Nvidia.dropna()

,Price,Volume,Trading_value,Daily_Return,30D MA,60D MA,Uper Band,Lower Band
Date,,,,,,,,
2019-04-30,45.25,36290000.0,1.642122e+09,0.009369,46.045000,42.538833,49.040913,43.049087
2019-05-01,45.12,35430000.0,1.598602e+09,-0.002873,46.084667,42.669167,48.994994,43.174339
2019-05-02,45.80,39470000.0,1.807726e+09,0.015071,46.158000,42.807667,48.916152,43.399848
2019-05-03,45.75,33680000.0,1.540860e+09,-0.001092,46.150333,42.932667,48.911810,43.388857
2019-05-06,44.96,42220000.0,1.898211e+09,-0.017268,46.169667,43.067833,48.887635,43.451698
...,...,...,...,...,...,...,...,...
2024-01-29,624.65,33900000.0,2.117564e+10,0.023496,532.578000,502.785500,633.341900,431.814100
2024-01-30,627.74,39600000.0,2.485850e+10,0.004947,537.386000,506.193667,642.145518,432.626482
2024-01-31,615.27,45070000.0,2.773022e+10,-0.019865,541.598333,509.197167,648.432558,434.764109


In [9]:
daily_return_std = Nvidia['Daily_Return'].std()
Nvidia['Normalized_Volatility'] = Nvidia['Daily_Return'] / daily_return_std

C:\Users\Admin\AppData\Local\Temp\ipykernel_10724\3940457310.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
Nivida_check_1=Nvidia[['Price','30D MA','60D MA']]
fig = px.line(data_frame=Nivida_check_1)
fig.show()
#Đường cam phái trên là uptrend đường cam phía dưới là dowmntrend

In [10]:
Nivida_check_2=Nvidia[['Uper Band','Price','30D MA','60D MA','Lower Band']]
fig = px.line(data_frame=Nivida_check_2)
fig.show()